# AGO AGE Distributed Collaboration Audit

Description: AGO AGE Distributed Collaboration Audit is used to audit collaborations configured in the connected AGO or AGE Organization. The Audit gathers basic information about the distributed collaboration, and it also audits the sync history for the configured collaborations.

Created on: 6/12/25

Purpose: The purpose of this notebook is to gather basic information about the distributed collaboration, and it also audits the sync history for the configured collaborations. Follow the steps outlined below. You can connect to AGO or AGE, and the notebook will audit the collaborations configured for the connected AGO or AGE organization. The notebook prompts the user for a folder to save excel files of the results. To audit the sync history, you define a date range, and the notebook will return records showing information about the attempts to synchronize the collaboration over that date range. There is a limit of 10 records, so it will return the most recent 10 records if the collaboration was synchronized more than 10 times over that date range. If a date range is not defined the sync history will not be audited. The results are displayed in a pandas data frame and exported to excel. You can optionally upload the collaboration audit to AGO or AGE. 

Authored By: Joe Guzi

Previous Production Date:       

Production Date: 6/12/25

Notes: AGO, AGE, Collaboration

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis import ItemProperties
from arcgis.gis import ItemTypeEnum
from arcgis.gis.admin import Collaboration
from arcgis.gis.admin import CollaborationManager
import datetime
import getpass
from IPython.display import display
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
import os
import ipywidgets as widgets
import pandas as pd
pd.options.display.max_rows = None
print("Import Libraries Complete!")
print("Define Variables")
PickedBeginningTimeEpoch = ""
PickedEndingTimeEpoch = ""
print("Define Variables Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Choose an option below to connect to ArcGIS Online or ArcGIS Enterprise

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables\n",
Organization = input("What's your oranization?")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password?")
# Connection
gis = GIS(Organization, User, Password, verify_cert=False)
print(gis)
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")"

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Audit Setup
Run the following sections to set up the Audit process. If a section is marked optional, you do not have to run that section.

## Functions

In [ ]:
print("Begin Function Section.")
# This is where you define Functions

def CheckDictionaryKey(Dictionary, Key):
    value = Dictionary.get(Key)
    #print(value)
    if value == -1:
        #print("negative1")
        return 0
    elif value is not None:
        return(value)
    else:
        return "none"

print("Function Section Complete!")

## Create a folder to save the reports

In [ ]:
filepath = asksaveasfilename(title="Select a location to create a folder")
print(filepath)
os.makedirs(filepath, exist_ok=True) # Creates folder and parent folders if needed
    

## Select Date Range (optional)
If you want to generate a sync report for the distrubuted collaborations, run these sections, and select a date range.
If nothing is selected, then the sync reports will not be generated

In [ ]:
BeginningTime = widgets.DatetimePicker(description='Select a Beginning Time', disabled=False)
BeginningTime

In [ ]:
EndingTime = widgets.DatetimePicker(description='Select an Ending Time', disabled=False)
EndingTime

In [ ]:
print("Selected Time Range")
PickedBeginningTime = BeginningTime.value
#print(PickedBeginningTime)
PickedBeginningTimeEpoch = PickedBeginningTime.timestamp()*1000
#print(PickedBeginningTimeEpoch)
#print(" <-----------> ")
PickedEndingTime = EndingTime.value
#print(PickedEndingTime)
PickedEndingTimeEpoch = PickedEndingTime.timestamp()*1000
#print(PickedEndingTimeEpoch)
print(str(PickedBeginningTime) + " <-----------> " + str(PickedEndingTime))

# I am the master of time

# Step 4: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
# this is where you put the code you want to execute.
t = ""
CollaborationAuditList = []
OrgName = gis.properties['name']
print("Current Organization: " + str(OrgName))
# Access Collaboration Manager
StarkGISPortalCollab = CollaborationManager(gis)
# Get the list of collaborations
collaborations = gis.admin.collaborations.list()
# Print the details of each collaboration
for collab in collaborations:
    SyncResultList = []
    #print(collab)
    CollaborationID = CheckDictionaryKey(collab, 'id')
    #print(f"ID: " + str(CollaborationID))
    CollaborationName = CheckDictionaryKey(collab, 'name')
    print(f"Name: " + str(CollaborationName))
    CollaborationDescription = CheckDictionaryKey(collab, 'description')
    #print(f"Description: " + str(CollaborationDescription))
    CollaborationHostPortalID = CheckDictionaryKey(collab, 'collaborationHostPortalId')
    #print(f"Host Portal ID: " + str(CollaborationHostPortalID))
    CollaborationOwner = CheckDictionaryKey(collab['config'], 'ownerUsername')
    #print(f"Collabration Owner: " + str(CollaborationOwner))
    CreatedEPOCH = CheckDictionaryKey(collab, 'created')
    #print(CreatedEPOCH)
    CollaborationCreatedDate = datetime.datetime.fromtimestamp(CreatedEPOCH / 1000)
    #print(f"Collabration Created Date: " + str(CollaborationCreatedDate))
    ModifiedEPOCH = CheckDictionaryKey(collab, 'modified')
    #print(ModifiedEPOCH)
    CollaborationModifiedDate = datetime.datetime.fromtimestamp(ModifiedEPOCH / 1000)
    #print(f"Collabration Modified Date: " + str(CollaborationModifiedDate))
    #print(f"Status: {collab.status}")
    
    CollaborationObject = Collaboration(StarkGISPortalCollab, collab.id)
    #print(CollaborationObject)
    CollaborationWorkspaces = CollaborationObject.workspaces
    #print(CollaborationWorkspaces)
    for CollabWorkspace in CollaborationWorkspaces:
        #print(CollabWorkspace)
        CollaborationWS_ID = CheckDictionaryKey(CollabWorkspace, 'id')
        #print("WS ID: " + str(CollaborationWS_ID))
        CollaborationWS_Name = CheckDictionaryKey(CollabWorkspace, 'name')
        #print("WS Name: " + str(CollaborationWS_Name))
        CollaborationWS_Description = CheckDictionaryKey(CollabWorkspace, 'description')
        #print("WS Description: " + str(CollaborationWS_Description))
        WSCreatedEPOCH = CheckDictionaryKey(CollabWorkspace, 'created')
        #print(WSCreatedEPOCH)
        CollaborationWS_CreatedDate = datetime.datetime.fromtimestamp(WSCreatedEPOCH / 1000)
        #print("WS Created: " + str(CollaborationWS_CreatedDate))
        WSModifiedEPOCH = CheckDictionaryKey(CollabWorkspace, 'modified')
        #print(WSModifiedEPOCH)
        CollaborationWS_ModifiedDate = datetime.datetime.fromtimestamp(WSModifiedEPOCH / 1000)
        #print("WS Modified: " + str(CollaborationWS_ModifiedDate))
        WSLastSyncSuccessEPOCH = CheckDictionaryKey(CollabWorkspace, 'lastSyncSuccess')
        CollaborationWS_LastSyncSuccessDate = datetime.datetime.fromtimestamp(WSLastSyncSuccessEPOCH / 1000)
        #print("WS LastSyncSuccess: " + str(CollaborationWS_LastSyncSuccessDate))
        CollaborationWS_Grouplinks = CollabWorkspace['participantGroupLinks'][0]
        CollaborationWS_GroupID = CheckDictionaryKey(CollaborationWS_Grouplinks, 'portalGroupId')
        #print("WS GroupID: " + str(CollaborationWS_GroupID))
        CollaborationWS_GroupName = CheckDictionaryKey(CollaborationWS_Grouplinks,'portalGroupName')
        #print("WS GroupName: " + str(CollaborationWS_GroupName))
        CollaborationWS_AccessMode = CheckDictionaryKey(CollaborationWS_Grouplinks, 'accessMode')
        #print("WS AccessMode: " + str(CollaborationWS_AccessMode)) 
        CollaborationDictionary = {}
        CollaborationDictionary["Organization"]= str(OrgName)
        CollaborationDictionary["CollaborationID"]= str(CollaborationID)
        CollaborationDictionary["CollaborationName"]= str(CollaborationName)
        CollaborationDictionary["CollaborationDecsription"]= str(CollaborationDescription)
        CollaborationDictionary["HostPortalID"]= str(CollaborationHostPortalID)
        CollaborationDictionary["CollaborationOwner"]= str(CollaborationOwner)
        CollaborationDictionary["CollaborationCreatedDate"]= str(CollaborationCreatedDate)
        CollaborationDictionary["CollaborationModified"]= str(CollaborationModifiedDate)
        CollaborationDictionary["WorkspaceID"]= str(CollaborationWS_ID)
        CollaborationDictionary["WorkspaceName"]= str(CollaborationWS_Name)
        CollaborationDictionary["WorkspaceDescription"]= str(CollaborationWS_Description)
        CollaborationDictionary["WorkspaceCreatedDate"]= str(CollaborationWS_CreatedDate)
        CollaborationDictionary["WorkspaceModifiedDate"]= str(CollaborationWS_ModifiedDate)
        CollaborationDictionary["WorkSpaceLastSyncSuccessDate"]= str(CollaborationWS_LastSyncSuccessDate)
        CollaborationDictionary["WorkspaceGroupID"]= (CollaborationWS_GroupID)
        CollaborationDictionary["WorkspaceGroupName"]= (CollaborationWS_GroupName)
        CollaborationDictionary["WorkspaceAccessMode"]= (CollaborationWS_AccessMode)
        CollaborationAuditList.append(CollaborationDictionary)
        # Sync audit section
        if PickedBeginningTimeEpoch != "" and PickedEndingTimeEpoch != "":
            print("Date Range Selected. Beggining Sync Audit.")
            WSsyncStatus = collab.sync_status(CollaborationWS_ID)
            # Note: Sync Status only returns a max of 10 records.
            #print(WSsyncStatus)
            for WSsyncJob in WSsyncStatus:
                #CheckDictionaryKey(Dictionary, Key)
                WSsyncJobStartTimeEPOCH = CheckDictionaryKey(WSsyncJob, 'startTime')
                #print(WSsyncJobStartTimeEPOCH)
                WSsyncJobStartTime = datetime.datetime.fromtimestamp(WSsyncJobStartTimeEPOCH / 1000)
                #print(WSsyncJobStartTime)
                WSsyncJobEndTimeEPOCH = CheckDictionaryKey(WSsyncJob, 'endTime')
                #print(WSsyncJobEndTimeEPOCH)
                WSsyncJobEndTime = datetime.datetime.fromtimestamp(WSsyncJobEndTimeEPOCH / 1000)
                #print(WSsyncJobEndTime)
                if PickedBeginningTimeEpoch <= WSsyncJobEndTimeEPOCH <= PickedEndingTimeEpoch:
                    #print(WSsyncJob)
                    WSsyncJob["startTime"] = WSsyncJobStartTime
                    WSsyncJob["endTime"] = WSsyncJobEndTime
                    SyncResultList.append(WSsyncJob)
            #add  dataframe code here
            #print(SyncResultList)
            df = pd.DataFrame(SyncResultList)
            display(df)
            #export dataframe to csv
            ExcelFile = filepath + "/" + CollaborationName + "SyncResults.xlsx"
            print(ExcelFile)
            df.to_excel(ExcelFile, index=False)
        else:
            print("No Date Range Selected. Continue.")
# Collaboration Audit
#print(CollaborationAuditList)
df = pd.DataFrame(CollaborationAuditList)
display(df)
#export dataframe to csv
CSVFile = filepath + "/" + "CollaborationAudit.csv"
print(CSVFile)
df.to_csv(CSVFile, index=False)

print("Execute Code Section Complete!") 

# Step 5: Upload to AGO (Optional)

In [ ]:
print("Begin Upload to AGO Section.") 
file = CSVFile.rpartition('/')
#print(file)
filename = file[2]
#print(file[2])
testfile = gis.content.search(query="title:"+filename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(CSVFile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    item_props = ItemProperties(title=filename + "_csv", item_type=ItemTypeEnum.CSV.value)
    print("Add csv to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=CSVFile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish csv as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The CSV has been published.\nAll done, thanks for playing!")